### example skript for estimating the reflectivity / absorbtion using the transfer-matrix method (TMM)

In [ ]:
import numpy as np

## @brief Calculate the reflectivity, transmittance, and absorption of a multilayer structure using the transfer matrix method (TMM)
#
# @details This function calculates the reflectivity, transmittance, and absorption of a multilayer structure using the TMM.
#          It takes into account the complex refractive indices and thicknesses of each layer.
#
# @param[in] wavelength The wavelength of the incident light in meters
# @param[in] n_list A list of complex refractive indices from the incident medium to the substrate
# @param[in] d_list A list of thicknesses in meters for each physical layer (excluding incident and substrate)
#
# @return A tuple containing the reflectivity (R), transmittance (T), and absorption per layer (A_layers)
#
# @note The absorption per layer is calculated using a crude approximation based on energy conservation.

def tmm_normal_incidence(wavelength, n_list, d_list):

    # incident index and substrate index
    n0 = n_list[0]
    ns = n_list[-1]

    # build characteristic matrices for layers 1..N (excluding incident and substrate)
    M = np.array([[1+0j, 0+0j],
                  [0+0j, 1+0j]])
    layer_indices = n_list[1:-1]
    for nj, dj in zip(layer_indices, d_list):
        delta = 2*np.pi*nj*dj / wavelength
        cosd = np.cos(delta)
        sind = np.sin(delta)
        Mj = np.array([[cosd, 1j*sind / nj],
                       [1j*nj*sind, cosd]])
        M = M.dot(Mj)

    # reflection coefficient
    num = (M[0,0] + M[0,1]*ns)*n0 - (M[1,0] + M[1,1]*ns)
    den = (M[0,0] + M[0,1]*ns)*n0 + (M[1,0] + M[1,1]*ns)
    r = num / den
    R = np.abs(r)**2

    # transmission amplitude (optional)
    t = 2*n0 / ((M[0,0] + M[0,1]*ns)*n0 + (M[1,0] + M[1,1]*ns))
    T = (np.abs(t)**2) * (ns.real / n0.real)  # power transmittance

    # approximate absorption per layer via energy conservation:
    A_total = 1.0 - R - T
    # For thin strongly absorbing layers you should compute internal fields to split A_total.
    # A crude split by imaginary index * thickness:
    im_vals = [ (nj.imag * dj) for nj,dj in zip(layer_indices, d_list) ]
    if np.sum(im_vals) == 0:
        absorptions = [0.0]*len(im_vals)
    else:
        absorptions = [A_total * v / np.sum(im_vals) for v in im_vals]

    return R, T, absorptions

# Example usage (fill with real n,k,d):
wavelength = 355e-9
# n_list: [air, Ni, Ge, substrate]
# fill with complex n + 1j*k values from literature for 355 nm
n_list = [1+0j, 1.73+2.33j, 4.02+2.66j, 1.5+0j]  # EXAMPLE only
d_list = [10e-9, 10e-9]  # Ni 10 nm, Ge 10 nm
R, T, A_layers = tmm_normal_incidence(wavelength, n_list, d_list)
print("R=", R, "T=", T, "A_layers=", A_layers, "A_total=", 1-R-T)


R= 2.3862276595289957 T= 0.7099357992616107 A_layers= [np.float64(-0.9787697112188601), np.float64(-1.117393747571746)] A_total= -2.096163458790606
